In [ ]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
import math
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/Machine_learning/CIFAR_DENSENET/")

In [ ]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [ ]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 150
compression = 0.5
dropout_rate = 0.2

In [ ]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
# normalize data
x_train = X_train.astype('float32') / 255
x_test = X_test.astype('float32') / 255

In [ ]:
X_train.shape

(50000, 32, 32, 3)

In [ ]:
X_test.shape

(10000, 32, 32, 3)

In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 10)


In [ ]:
data_augmentation = True

In [ ]:
# Dense Block
def denseblock(input, num_filter, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(L): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same', kernel_initializer='he_normal')(relu)
        if dropout_rate > 0 and data_augmentation == False:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same', kernel_initializer='he_normal')(relu)
    if dropout_rate > 0 and data_augmentation == False:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
growth_rate = 12
L = 16
dropout_rate = 0.2

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D((2*growth_rate), (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, (2*growth_rate), dropout_rate)
First_Transition = transition(First_Block, (4*growth_rate), dropout_rate)

Second_Block = denseblock(First_Transition, (2*growth_rate), dropout_rate)
Second_Transition = transition(Second_Block, (4*growth_rate), dropout_rate)

Third_Block = denseblock(Second_Transition, (2*growth_rate), dropout_rate)
Third_Transition = transition(Third_Block, (4*growth_rate), dropout_rate)

Last_Block = denseblock(Third_Transition,(2*growth_rate), dropout_rate)
output = output_layer(Last_Block)

In [ ]:
def lr_schedule(epoch):
  lr = 0.001
  if epoch >= 25 and epoch < 50:
    lr *= 1e-1
  elif epoch >= 50 and epoch < 75:
    lr *= 1e-2
  elif epoch >=75 and epoch < 100:
    lr *= 1e-3
  elif epoch >= 100 and epoch < 125:
    lr *= 0.5e-2
  elif epoch >= 125 and epoch <= 150:
    lr *= 0.5e-3
  print("Learning Rate is :\n", lr)
  return lr

    
lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = np.sqrt(0.1), patience = 5, min_lr = 1e-7, verbose = 1)

filepath = "Saved_model/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
model_chkpt = ModelCheckpoint(filepath = filepath, monitor = "val_accuracy", save_best_only=True, verbose = 1)

callbacks = [lr_scheduler, reduce_lr, model_chkpt]

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 24)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 24)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
print(len(model.layers))

275


In [ ]:
# Loss function & Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
# Data Augmentation and model training
if not data_augmentation:
  print('Not using data augmentation.')
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test), callbacks= callbacks)
  
else:
  print('Using real-time data augmentation.')
  # preprocessing  and realtime data augmentation
  augment = ImageDataGenerator(
      rotation_range=20,  # randomly rotate images in the range (deg 0 to 180)
      width_shift_range=0.15,  # randomly shift images horizontally
      height_shift_range=0.15,  # randomly shift images vertically
      horizontal_flip=True,  # randomly flip images
      zoom_range=0.2)  # zoom images

  augment.fit(X_train)

  steps_per_epoch = math.ceil(len(X_train)/ batch_size)
  model.fit(augment.flow(X_train, y_train, batch_size= batch_size), verbose=1, validation_data=(X_test,y_test), epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks= callbacks)

Using real-time data augmentation.
Learning Rate is :
 0.001

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/150
1563/1563 [==============================] - ETA: 0s - loss: 1.5548 - accuracy: 0.4344
Epoch 00001: val_accuracy improved from -inf to 0.41030, saving model to Saved_model/weights-01-0.4103.hdf5
1563/1563 [==============================] - 123s 79ms/step - loss: 1.5548 - accuracy: 0.4344 - val_loss: 1.9029 - val_accuracy: 0.4103
Learning Rate is :
 0.001

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/150
1563/1563 [==============================] - ETA: 0s - loss: 1.1581 - accuracy: 0.5860
Epoch 00002: val_accuracy improved from 0.41030 to 0.66530, saving model to Saved_model/weights-02-0.6653.hdf5
1563/1563 [==============================] - 121s 78ms/step - loss: 1.1581 - accuracy: 0.5860 - val_loss: 0.9552 - val_accuracy: 0.6653
Learning Rate is :
 0.001

Epoch 00003: LearningRateScheduler reducing learning rate t

In [ ]:
#model.load_weights("DNST_model.h5")

In [ ]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 6s 20ms/step - loss: 0.2813 - accuracy: 0.9130
Test loss: 0.2812565565109253
Test accuracy: 0.9129999876022339


In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
